In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [94]:
# import warnings
# warnings.filterwarnings("ignore")
books = pd.read_csv("./data/Books.csv")
ratings = pd.read_csv("./data/Ratings.csv")
users = pd.read_csv("./data/Users.csv")

In [95]:
x=ratings['User-ID'].value_counts()>200


In [96]:
y=x[x].index


In [97]:
ratings=ratings[ratings['User-ID'].isin(y)]


In [98]:
ratings_with_books=ratings.merge(books,on='ISBN')


In [99]:
number_rating=ratings_with_books.groupby('Book-Title')['Book-Rating'].count().reset_index()


In [100]:
number_rating.rename(columns={'Book-Rating':'number of ratings'},inplace=True)


In [101]:
final_rating=ratings_with_books.merge(number_rating,on='Book-Title')


In [102]:
final_rating=final_rating[final_rating['number of ratings']>=50]


In [103]:
final_rating


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,number of ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
...,...,...,...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,50


In [104]:
final_rating.drop_duplicates(['User-ID','Book-Title'],inplace=True)


In [105]:
rating_pivot=final_rating.pivot_table(columns='User-ID',index='Book-Title',values='Book-Rating')


In [106]:
rating_pivot.fillna(0,inplace=True)


In [107]:
rating_pivot.head(2)


User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(rating_pivot)

In [109]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')

In [110]:
model.fit(book_sparse)


NearestNeighbors(algorithm='brute')

In [111]:
distances,suggestions=model.kneighbors(rating_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)


In [112]:
from joblib import dump

# model variable is named 'model'
dump(model, 'model.joblib')

['model.joblib']

In [113]:
import pickle

# matrix variable is named 'rating_pivot'
with open('rating_pivot.pickle', 'wb') as f:
    pickle.dump(rating_pivot, f)
